<a href="https://colab.research.google.com/github/simon-bouchard/freecodecamp-machine-learning-models/blob/main/Copie_de_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

from imblearn.over_sampling import ADASYN

import numpy as np
import matplotlib.pyplot as plt


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#get data
train_data = pd.read_csv('train-data.tsv', sep='\t', header=None)
valid_data = pd.read_csv('valid-data.tsv', sep='\t', header=None)

train_labels = train_data[0]
train_texts = train_data[1]

valid_labels = valid_data[0]
valid_texts = valid_data[1]

train_labels.shape

In [ ]:
#Preprocessing

label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
valid_labels = label_encoder.transform(valid_labels)

tokenizer = Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(train_texts)

num_words = len(tokenizer.word_index)
max_len = max(max(len(text.split()) for text in train_texts),
              max(len(text.split()) for text in valid_texts))

train_sequences = tokenizer.texts_to_sequences(train_texts)
train_sequences = pad_sequences(train_sequences, padding='post', maxlen=max_len)

valid_sequences = tokenizer.texts_to_sequences(valid_texts)
valid_sequences = pad_sequences(valid_sequences, padding='post', maxlen=max_len)

In [ ]:
#model
inputs = Input(shape=(max_len,))
x = Embedding(input_dim=num_words, output_dim=32, input_length=max_len)(inputs)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = Bidirectional(LSTM(32))(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

In [ ]:
#Train model
#Set class weight (there are more ham than spam)(not necessary)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

class_weights = dict(enumerate(class_weights))
print(f"Class Weights: {class_weights}")

history = model.fit(
    train_sequences,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(valid_sequences, valid_labels),
    class_weight=class_weights
    )

model.save('spam_ham_model.keras')

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

#Uncomment if you don't want to run the previous cells
#model = load_model('spam_ham_model.keras')

def predict_message(pred_text):

  sequence = tokenizer.texts_to_sequences([pred_text])
  sequence = pad_sequences(sequence, padding='post', maxlen=max_len)

  predictions = model.predict(sequence)

  prediction = [predictions[0][0]]

  prediction.append('ham' if predictions[0][0] <= 0.5 else 'spam')

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
